### Parsing SLAC Fe55 Analysis files
- Craig Lage - 28-Oct-19

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys, glob, time, scipy
from scipy import stats
import pickle as pkl
from astropy.io import fits as pf

In [ ]:
# Now get Fe55 stamps
datadir = '/home/cslage/Research/LSST/code/poisson/fe55_data/'
E2VfitsFile = datadir+'R21_S10_6801D_psf_results_nsig4.fits'
E2VgainFile = datadir+'R21_S10_6801D_eotest_results.fits'
ITLfitsFile = datadir+'R20_S10_6801D_psf_results_nsig4.fits'
ITLgainFile = datadir+'R20_S10_6801D_eotest_results.fits'
simdir = '/home/cslage/Research/LSST/code/poisson/Poisson_CCD_Fe55_1'
#E2Vsimdir = simdir+'/cc_e2v_noCorrection'; numSpots = 128; DiffMult = 1.0
#E2Vsimdir = simdir+'/cc_e2v_2p23'; numSpots = 1024; DiffMult = 2.23
E2Vsimdir = simdir+'/cc_e2v_2p30'; numSpots = 1024; DiffMult = 2.30
ITLsimdir = simdir+'/cc_itl_2p30'; numSpots = 1024; DiffMult = 2.30

In [ ]:
# Classes to put the data
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=np.linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=np.linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=np.zeros([nx,ny,nstamps])
        self.gain = 1.0

def ReadCCFile(filename, Nx, Ny):                                                                                                      
    file = open(filename,'r')                                                                                                          
    lines=file.readlines()                                                                                                             
    file.close()                                                                                                                       
    lines.remove(lines[0]) # Strip the title line                                                                                      
    elec = np.zeros([Nx,Ny])                                                                                                           
    count = 0                                                                                                                          
    for line in lines:                                                                                                                 
        items = line.split()                                                                                                           
        i = int(items[0]) - 1                                                                                                          
        j = int(items[1]) - 1                                                                                                          
        elec[i,j] = int(items[2])                                                                                                      
                                                                                                                                       
    return elec                                                                                                                        

def BuildDataSpotList(fitsfilename, gainfilename, ampnumber, nx, ny):
    dy = 7; dx = 9 # Size of stamp in fitsfile catalog
    Kalpha = 1620 # Electrons in Kalpha line
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    hdulist=pf.open(fitsfilename)
    data=hdulist[ampnumber].data
    gain_hdulist=pf.open(gainfilename)
    gain = gain_hdulist[1].data['GAIN'][ampnumber-1]
    numRows = data.shape[0]
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numRows)
    spotlist.gain = gain
    mincounts = Kalpha * 0.95
    maxcounts = Kalpha * 1.05
    spotCounter = 0
    for row in range(numRows):
        prect_data = data[row]['PRECT_DATA']
        #print(row, prect_data.max())
        spot = np.zeros([nx,ny])
        for i in range(nx):
            for j in range(ny):
                spot[j,i] = prect_data[(i+1) + dx * (j+1)] * gain
        count = spot.sum()
        if count > mincounts and count < maxcounts:
            spotlist.data[:,:,spotCounter] = spot
            spotCounter += 1

    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,spotCounter)
    newspotlist.data = spotlist.data[:,:,0:spotCounter]
    newspotlist.gain = spotlist.gain
    del spotlist
    return newspotlist

def BuildSimSpotList(simdir, outputfilebase, nx, ny, numSpots):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numSpots)
    for spot in range(numSpots):
        cc_file = simdir+'/run_%d/%s_0_CC.dat'%(spot,outputfilebase)
        elec = ReadCCFile(cc_file, nx, ny)
        for i in range(nx):
            for j in range(ny):
                spotlist.data[i,j,spot] = elec[i,j]
    return spotlist


In [ ]:
nx = 5; ny = 5; nAmps = 16; 
AveCounts = np.zeros([nx,ny,nAmps])
TotalSpots = 0
for amp in range(nAmps):
    spotlist = BuildDataSpotList(E2VfitsFile, E2VgainFile, amp+1, nx, ny)
    AmpAve = spotlist.data.sum(axis=2) / float(spotlist.nstamps)
    AveCounts[:,:,amp] = AmpAve
    TotalSpots += spotlist.nstamps



In [ ]:
MeasAve = AveCounts.mean(axis=2)
MeasStd = AveCounts.std(axis=2)
simData = BuildSimSpotList(E2Vsimdir, "Pixel", nx, ny, numSpots)    
SimAve = simData.data.mean(axis=2)
SimStd = simData.data.std(axis=2) / np.sqrt(simData.nstamps)
testData = simData.data[2,2,:]
StdCombined = np.sqrt(SimStd*SimStd + MeasStd * MeasStd) + 0.5 # Added in 0.5 electrons noise to the error
error = (MeasAve - SimAve) * (MeasAve - SimAve) / (StdCombined*StdCombined)
print("Chi^2 = %.2f"%(error.sum()/(nx*ny-1.0)))

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,1,1,aspect = 1)
plt.suptitle("Stacked Fe55 events - E2V, %d hits\n%d Simulated events - DiffMultiplier = %.2f"%(TotalSpots,numSpots, DiffMult), fontsize = 24)

for i in range(spotlist.nx):
    plt.plot([spotlist.x[i]-0.5, spotlist.x[i]-0.5], [spotlist.y[0]-0.5, spotlist.y[-1]+0.5], color = 'black',
             ls = '--')
    plt.plot([spotlist.x[i]+0.5, spotlist.x[i]+0.5], [spotlist.y[0]-0.5, spotlist.y[-1]+0.5], color = 'black',
             ls = '--')
for j in range(spotlist.ny):
    plt.plot([spotlist.x[0]-0.5, spotlist.x[-1]+0.5], [spotlist.y[j]-0.5, spotlist.y[j]-0.5], color = 'black',
             ls = '--')
    plt.plot([spotlist.x[0]-0.5, spotlist.x[-1]+0.5], [spotlist.y[j]+0.5, spotlist.y[j]+0.5], color = 'black',
             ls = '--')


for i in range(spotlist.nx):
    for j in range(spotlist.ny):
        plt.text(spotlist.x[i] - 0.35, spotlist.y[j] + 0.05, 
                 "%.1f\n +/- %.1f"%(MeasAve[i,j], MeasStd[i,j]), color='green', weight='bold', fontsize=16)
        plt.text(spotlist.x[i] - 0.35, spotlist.y[j] - 0.35, 
                 "%.1f\n +/- %.1f"%(SimAve[i,j], SimStd[i,j]), color='red', weight='bold', fontsize=16)
plt.xticks([-2,-1,0,1,2], fontsize=18)
plt.yticks([-2,-1,0,1,2], fontsize=18)
print(AveCounts.sum() / nAmps)
plt.savefig(datadir+'E2V_Fe55_DM2p30_31Oct19.pdf')

In [ ]:
nx = 5; ny = 5; nAmps = 10; 
AveCounts = np.zeros([nx,ny,nAmps])
TotalSpots = 0
for amp in range(nAmps):
    spotlist = BuildDataSpotList(ITLfitsFile, ITLgainFile, amp+1, nx, ny)
    AmpAve = spotlist.data.sum(axis=2) / float(spotlist.nstamps)
    AveCounts[:,:,amp] = AmpAve
    TotalSpots += spotlist.nstamps

In [ ]:
MeasAve = AveCounts.mean(axis=2)
MeasStd = AveCounts.std(axis=2)
simData = BuildSimSpotList(ITLsimdir, "Pixel", nx, ny, numSpots)    
SimAve = simData.data.mean(axis=2)
SimStd = simData.data.std(axis=2) / np.sqrt(simData.nstamps)
testData = simData.data[2,2,:]
StdCombined = np.sqrt(SimStd*SimStd + MeasStd * MeasStd) + 0.5 # Added in 0.5 electrons noise to the error
error = (MeasAve - SimAve) * (MeasAve - SimAve) / (StdCombined*StdCombined)
print("Chi^2 = %.2f"%(error.sum()/(nx*ny-1.0)))

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,1,1,aspect = 1)
plt.suptitle("Stacked Fe55 events - ITL, %d hits\n%d Simulated events - DiffMultiplier = %.2f"%(TotalSpots,numSpots, DiffMult), fontsize = 24)

for i in range(spotlist.nx):
    plt.plot([spotlist.x[i]-0.5, spotlist.x[i]-0.5], [spotlist.y[0]-0.5, spotlist.y[-1]+0.5], color = 'black',
             ls = '--')
    plt.plot([spotlist.x[i]+0.5, spotlist.x[i]+0.5], [spotlist.y[0]-0.5, spotlist.y[-1]+0.5], color = 'black',
             ls = '--')
for j in range(spotlist.ny):
    plt.plot([spotlist.x[0]-0.5, spotlist.x[-1]+0.5], [spotlist.y[j]-0.5, spotlist.y[j]-0.5], color = 'black',
             ls = '--')
    plt.plot([spotlist.x[0]-0.5, spotlist.x[-1]+0.5], [spotlist.y[j]+0.5, spotlist.y[j]+0.5], color = 'black',
             ls = '--')


for i in range(spotlist.nx):
    for j in range(spotlist.ny):
        plt.text(spotlist.x[i] - 0.35, spotlist.y[j] + 0.05, 
                 "%.1f\n +/- %.1f"%(AveCounts[i,j,:].mean(), AveCounts[i,j,:].std()), color='green', weight='bold', fontsize=16)
        plt.text(spotlist.x[i] - 0.35, spotlist.y[j] - 0.35, 
                 "%.1f\n +/- %.1f"%(SimAve[i,j], SimStd[i,j]), color='red', weight='bold', fontsize=16)
plt.xticks([-2,-1,0,1,2], fontsize=18)
plt.yticks([-2,-1,0,1,2], fontsize=18)
print(AveCounts.sum() / nAmps)
plt.savefig(datadir+'ITL_Fe55_DM2p30_31Oct19.pdf')